In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import MaxoutDense
from keras.layers import LocallyConnected2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import MaxoutDense
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')



In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load data
(X_train_1, y_train_1), (X_test_1, y_test_1) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train_1 = X_train_1.reshape(X_train_1.shape[0], 1, 28, 28).astype('float32')
X_test_1 = X_test_1.reshape(X_test_1.shape[0], 1, 28, 28).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train_1 = X_train_1 / 255
X_test_1 = X_test_1 / 255

In [3]:
five_numbers = np.array([0, 3, 4, 5, 7])
mask_train = np.in1d(y_train_1, five_numbers)
mask_test  = np.in1d(y_test_1, five_numbers)
mask_train[:5]

In [4]:
X_03457_train = X_train_1[mask_train]
y_03457_train = y_train_1[mask_train]

X_03457_test = X_test_1[mask_test]
y_03457_test = y_test_1[mask_test]

X_03457_train = X_03457_train.reshape((X_03457_train.shape[0], X_03457_train.shape[2], X_03457_train.shape[3]))
X_03457_test = X_03457_test.reshape((X_03457_test.shape[0], X_03457_test.shape[2], X_03457_test.shape[3]))

del X_train_1
del y_train_1
del X_test_1
del y_test_1

print(X_03457_train.shape)

In [5]:
for i in range(4):
  plt.subplot(221 + i)
  plt.imshow(X_03457_train[i].reshape(28, 28), cmap=plt.get_cmap('gray'))
  plt.title('{}'.format(y_03457_train[i]), fontsize=14)
  plt.axis('off')
  # show the plot
display()

In [6]:
# one hot encode outputs
y_03457_train = np_utils.to_categorical(y_03457_train)[:, five_numbers]
y_03457_test = np_utils.to_categorical(y_03457_test)[:, five_numbers]

In [7]:
for i in range(4):
  plt.subplot(221 + i)
  plt.imshow(X_03457_train[i].reshape(28, 28), cmap=plt.get_cmap('gray'))
  plt.title('{}'.format(y_03457_train[i]), fontsize=14)
  plt.axis('off')
  # show the plot
display()

In [9]:
joined_train = np.hstack((X_03457_train.reshape((-1, 784)), y_03457_train))
joined_test = np.hstack((X_03457_test.reshape((-1, 784)), y_03457_test))

iterations = 5
data_train = np.copy(joined_train)
data_test = np.copy(joined_test)

for i in range(iterations): 
  np.random.shuffle(joined_train)
  np.random.shuffle(joined_test)
  data_train = np.vstack((data_train, joined_train))
  data_test = np.vstack((data_test, joined_test))

print(data_train.shape)
print(data_test.shape)


del X_03457_train
del y_03457_train

del X_03457_test
del y_03457_test

del joined_train
del joined_test


In [10]:
def make_3_digits(data):
  X = data[:, :784]
  y = data[:, 784:]
  
  i = 0
  N = X.shape[0] - 3
  num_features = 3 * X.shape[1]
  num_classes  = 3 * y.shape[1]

  X_3 = np.zeros((N, num_features))
  y_3 = np.zeros((N, num_classes ))
  
  while i < N:
      num1 = X[i].reshape((28, 28))
      num2 = X[i+1].reshape((28, 28))
      num3 = X[i+2].reshape((28, 28))
      num123 = np.hstack((num1, num2, num3))

      label_1 = y[i] 
      label_2 = y[i + 1]
      label_3 = y[i + 2]
      label_123 = np.hstack((label_1, label_2, label_3))
      
      X_3[i] = num123.ravel()      
      y_3[i] = label_123.ravel()
      
      i+=1
      
  return X_3, y_3



In [11]:
X_train_3, y_train_3 = make_3_digits(data_train)
X_test_3, y_test_3 = make_3_digits(data_test)

print(X_train_3.shape)
print(y_train_3.shape)

print('\n')

print(X_test_3.shape)
print(y_test_3.shape)

del data_train
del data_test

In [12]:
for i in range(4):
  plt.subplot(221 + i)
  plt.imshow(X_train_3[i].reshape(28, 84), cmap=plt.get_cmap('gray'))
  plt.title('{}'.format(y_train_3[i]), fontsize=6)
  plt.axis('off')
  # show the plot
display()

In [13]:
def remove_digit(digit, three_digit):
    two_digit = np.copy(three_digit)
    two_digit.reshape(28, 84)[:, (digit * 28) : ((digit + 1) * 28)] = 0
    return two_digit.ravel()

def revise_label(digit, label):
    label_copy = np.copy(label)
    label_copy[digit * 5 : (digit + 1) * 5] = 0
    return label_copy
    

remove_key = {0:'first', 1:'second', 2:'third'}

fig, axes = plt.subplots(1, 3, figsize=(20, 3))
for digit, ax in zip(np.arange(3), axes):
    new_digits = remove_digit(digit, X_train_3[0])    
    new_label = revise_label(digit, y_train_3[0])
    
    ax.imshow(new_digits.reshape(28, 84), cmap=plt.get_cmap('gray'))
    ax.set_title("{} digit removed".format(remove_key[digit]))
    ax.set_xlabel('Label\n {}'.format(new_label))
display(fig)    

In [14]:
def insert_spaces(X, y, p=0.5):
  N = X.shape[0]
  sample_size = int(N * p)
  space_idx = np.random.RandomState(1234).choice(np.arange(N), size=sample_size, replace=False)
  
  for i in space_idx:
    num_digits_to_remove = np.random.randint(1, 3)
    digit_places = [0, 1, 2]  
    j = 0
    while j < num_digits_to_remove:
        digit_to_remove = np.random.choice(digit_places)
        X[i] = remove_digit(digit_to_remove, X[i])
        y[i] = revise_label(digit_to_remove, y[i])
        digit_places.remove(digit_to_remove)
        j += 1
        
  return X, y

In [15]:
X_train_3_space, y_train_3_space = insert_spaces(X_train_3, y_train_3)
X_test_3_space, y_test_3_space = insert_spaces(X_test_3, y_test_3)

del X_train_3
del y_train_3
del X_test_3
del y_test_3

In [16]:
fig, axes = plt.subplots(5, 5, figsize=(20, 10))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(X_train_3_space[i].reshape(28, 84), cmap=plt.get_cmap('gray'))
    ax.set_title("{}".format(y_train_3_space[i]), fontsize=8)
    ax.set_axis_off()
display(fig)

In [17]:
# Prepare inputs for Keras
X_train = X_train_3_space.reshape((X_train_3_space.shape[0], 1, 28, 84))
y_train = y_train_3_space

X_test = X_test_3_space.reshape((X_test_3_space.shape[0], 1, 28, 84))
y_test = y_test_3_space

del X_train_3_space
del y_train_3_space

del X_test_3_space
del y_test_3_space

In [18]:
# fix random seed for reproducibility
seed = 1234
np.random.seed(seed)
num_classes = y_train.shape[1]

In [19]:
def larger_model():
	# create model
	model = Sequential()
	model.add(Convolution2D(30, 5, 5, border_mode='valid', input_shape=(1, 28, 84), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Convolution2D(15, 3, 3, activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [20]:
# x = Input((1, 28, 84))

# y = Convolution2D(32, 3, 3, activation="relu", border_mode="same")(x)
# y = Convolution2D(32, 3, 3, activation="relu")(y)
# y = MaxPooling2D((2, 2))(y)
# y = Dropout(0.25)(y)

# y = Convolution2D(64, 3, 3, border_mode="same", activation="relu")(y)
# y = Convolution2D(64, 3, 3, activation="relu")(y)
# y = MaxPooling2D((2, 2))(y)
# y = Dropout(0.25)(y)


# y = Flatten()(y)
# y = Dense(1024, activation="relu")(y)

# digits = Dense(15, activation="softmax")(y)


# model = Model(input=x, output=[digits])
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# def larger_model():
#   # create model
#   model = Sequential()

#   model.add(Convolution2D(48, 5, 5, border_mode='valid', input_shape=(1, 28, 84), activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))

#   model.add(Convolution2D(64, 5, 5, activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))
#   model.add(Dropout(0.2))

#   model.add(Convolution2D(128, 5, 5, activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), stride=(2, 1)))
#   model.add(Dropout(0.2))

#   model.add(Convolution2D(160, 5, 5, activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))
#   model.add(Dropout(0.2))

#   model.add(Convolution2D(192, 5, 5, activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))
#   model.add(Dropout(0.2))

#   model.add(Convolution2D(192, 5, 5, activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))
#   model.add(Dropout(0.2))

#   model.add(Convolution2D(192, 5, 5, activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))
#   model.add(Dropout(0.2))

#   model.add(Convolution2D(192, 3, 3, activation='relu'))
#   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 1)))
#   model.add(Dropout(0.2))

#   model.add(Flatten())
#   model.add(Dense(128, activation='relu'))
#   model.add(Dense(128, activation='relu'))
#   model.add(Dense(50, activation='relu'))
#   model.add(Dense(num_classes, activation='softmax'))
#   # Compile model
#   model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#   return model

In [22]:
# build the model
model = larger_model()
# Fit the model
model.fit(X_train[:35000], y_train[:35000], validation_data=(X_test, y_test), nb_epoch=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

In [23]:
print('done')